# Java Client Issue: Event Loop Exception
Reproducible case for event loop exception. How to get event loops working in the docker container?
<br>
This notebook requires Aerospike datbase running on localhost. Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) for additional details and the docker container.

## Ensure database is running
This notebook requires that Aerospike datbase is running.

In [63]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);
%sh asd

## Download Aerospike client from POM
Here the Java client version 5.0.0 is installed.

In [64]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
</dependencies>

## Connect to database and populate test data
The test data has ten records with user-key "id1-10", two bins (fields) "bin1" and "bin2", in the namespace "test" and set "test_small". 

In [65]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.policy.WritePolicy;

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

String Namespace = "test";
String SmallSet = "test_small";

WritePolicy wpolicy = new WritePolicy();
wpolicy.sendKey = true;
for (int i = 1; i <= 10; i++) {
    Key key = new Key(Namespace, SmallSet, "id-"+i);
    Bin bin1 = new Bin(new String("bin1"), i);
    Bin bin2 = new Bin(new String("bin2"), 1000+i);
    client.put(wpolicy, key, bin1, bin2);
}

System.out.format("Test data popuated");;

Initialized the client and connected to the cluster.
Test data popuated

# Issue: package io.netty.channel does not exist
As a result cannot use EventLoopGroup variants. 

In [66]:
import io.netty.channel.EventLoopGroup;
import io.netty.channel.epoll.EpollEventLoopGroup;
import io.netty.channel.nio.NioEventLoopGroup;

CompilationException: 

# Issue: Event Loop Throws a Null Pointer Exception.

Using an event loop without EventLoopGroup ("DIRECT_NIO" variant). Fails with the null pointer exception. 

In [67]:
// background execution
import com.aerospike.client.async.EventLoop;
import com.aerospike.client.async.EventLoops;
import com.aerospike.client.async.EventPolicy;
import com.aerospike.client.async.NettyEventLoops;
import com.aerospike.client.async.NioEventLoops;
import com.aerospike.client.listener.RecordListener;
import com.aerospike.client.util.Util;

// a simple record listener - prints out the key and bins
public class MyRecordListener implements RecordListener {
    public void onSuccess(Key key, Record record) throws AerospikeException {
        System.out.format("set=%s key=%s bins=%s\n", key.setName, key.userKey, record.bins);
    }
    public void onFailure(AerospikeException e) {
        System.out.format("Query failed: " + Util.getErrorMessage(e));
    }
}

EventLoops eventLoops = new NioEventLoops(new EventPolicy(), 1);
EventLoop eventLoop = eventLoops.get(0);

//execute get in background
Key key = new Key(Namespace, SmallSet, "id-5");
client.get(eventLoop, new MyRecordListener(), null, key);

EvalException: null

## Clean up

In [68]:
client.truncate(null, Namespace, null, null);
client.close();
System.out.println("Removed data and closed server connection.");

Removed data and closed server connection.


## Next steps

Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) to run additional Aerospike notebooks. To run a different notebook, download the notebook from the repo to your local machine, and then click on File->Open, and select Upload.